In [ ]:
#Loading of libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import warnings

Hello to everyone,

With the work I did this week, I will do a classification project without building any models. So how will you do it? We haven't heard of such a thing! We will do it using only means, medians and quartiles. So how? Come on let's see :)

**What would this value set be if a threshold was requested in Churn?**

In [ ]:
#Loading the data set
df = pd.read_csv("../input/churn-123/churn kopyas.csv")

#observation of the first 2 lines
df.head(2)

#Percentage of churn class found in data
(2037) / (7963 + 2037) 

In [ ]:
#We see how many observations each class passes in the data.
df.Exited.value_counts()

We dropped some variables from our dataset with the code snippet below. The reason for this is; The id and name variables do not have any positive effect on the model we will install.

In [ ]:
#Dropping unrelated variables
need_drops = ["RowNumber",  "Surname","CustomerId"]
df.drop(need_drops, axis=1, inplace=True)

In [ ]:
#Questioning of missing values. It does not have any missing value.
df.isnull().sum()

We have done a normality test below. This is very important to us. Because it will affect the success of the model. This is the test we did; shows whether the variables (columns) are distributed normally. Interpreting the results of our transaction; H0 is rejected because the pvalue values ​​of all variables are less than 0.05 in the normality tests. So all variables are not distributed normally.

In [ ]:
#Test of normality

for i in ['CreditScore' , 'Age', 'Balance', 'EstimatedSalary']:
    print(i)
    t_test , p_value = shapiro(df[i])
    print('Test İstatistiği = %.4f, p-değeri = %.4f' % (t_test, p_value))

With the one_hot_encoder function below; It is the conversion of non-numeric variables in the data to numerical values ​​(0 and 1). This is the language the machine will understand.

In [ ]:
def one_hot_encoder(dataframe, category_freq=10, nan_as_category=False):
    categorical_cols = [col for col in dataframe.columns if len(dataframe[col].value_counts()) < category_freq
                        and dataframe[col].dtypes == 'O']

    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, dummy_na=nan_as_category, drop_first=True)

    return dataframe

df = one_hot_encoder(df)

#first two observations
df.head(2)


With histogram, the distortions in the data are visually examined.

In [ ]:
#Drawing histogram
def hist_for_nums(data, numeric_cols):
    col_counter = 0
    data = data.copy()
    for col in numeric_cols:
        data[col].hist(bins=20)
        plt.xlabel(col)
        plt.title(col)
        plt.show()
        col_counter += 1
    print(col_counter, "variables have been plotted")


#Drawing a histogram for numerical variables.

a = ['CreditScore' , 'Age', 'Balance', 'EstimatedSalary']

hist_for_nums(df, a)  

With the violin graph, we analyzed the states of the variables in the data according to the target variable.

In [ ]:
for i in a:
    sns.catplot(x ="Exited" , y= i ,data = df , kind="violin")
    plt.show()

In [ ]:
#Calculating correlations
df.corr().T

# MODELLEME

In [ ]:
#In order to prepare for the model, we distinguish between the target variable and our independent values.
X = df.drop(['Exited'], axis=1)
y = df[["Exited"]]

#Dividing the data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify = y , random_state=46)


In [ ]:
# LightGBM

#Create a model
lgbm = LGBMClassifier(random_state=12345)
cross_val_score(lgbm, X, y, cv=10).mean()

# model tuning
lgbm_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [500, 1000],
               "max_depth": [3, 5]}

#Calculation of GridSearchCV
gs_cv = GridSearchCV(lgbm,
                     lgbm_params,
                     cv=5,
                     n_jobs=-1,
                     verbose=2).fit(X, y)


The graphic created below is very important to us. Because it shows the importance levels of the variables. On the other hand, we will select 3 variables with high importance and perform a completely manual classification process.

In [ ]:
#Building models with the best parameters
lgbm_tuned = LGBMClassifier(**gs_cv.best_params_).fit(X_train, y_train)
cross_val_score(lgbm_tuned, X_test, y_test, cv=10).mean()


feature_imp = pd.Series(lgbm_tuned.feature_importances_,
                        index=X.columns).sort_values(ascending=False)

#Graphical representation of variable significance levels
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Değişken Önem Skorları')
plt.ylabel('Değişkenler')
plt.title("Değişken Önem Düzeyleri")
plt.show()

#cv operations (crossing)
kfold = KFold(n_splits=10, random_state=123456)
cv_results = cross_val_score(LGBMClassifier(), X_train, y_train, cv=kfold, scoring="accuracy")


In the Lightgbm algorithm, we select the variables that are most important to us with FeatureImportance option. These variables are Age, Balance and EstimatedSalary.

In [ ]:
#Choosing high level of importance variables
df = df[["Age","Balance","EstimatedSalary","Exited"]]
df.head()

Now we might think that why do we create a threshold value?

This is because; It is the process of determining the limit to separate the classes of each variable and classifying them manually according to these values. Now the following may come to mind; Why did you use median instead of mean? This is because; We did the normality test at first and found that it was not distributed normally. When the data is not distributed normally, an intervening large value causes the mean to deviate. For this, I took the median value. But I didn't do this classically by taking the median of a variable. I did it like this; I found the median values ​​of classes 0 and 1 and by taking the average of them, I reached a healthier result. This is the average I get; it created the threshold value that initially demarcated my data. So he separated the 0 and 1 classes. But the threshold process is not over yet. Lets continue :)

In [ ]:
#Finding the threshold value according to the age variable
df.groupby("Exited").agg({"Age":"median"}).values.mean()


In [ ]:
#Finding the threshold value according to the balance variable
df.groupby("Exited").agg({"Balance":"median"}).values.mean()

In [ ]:
#Finding threshold value according to EstimatedSalary variable
df.groupby("Exited").agg({"EstimatedSalary":"median"}).values.mean()

The 3 cells above formed our first threshold value. We could fill in according to these 3 thresholds and get a low classification success. But here we have to pay attention to this; there is an imbalance between classes in the data. For this, we will use the quartiles and use the values ​​obtained as threshold values. So we will do a classification process by using two threshold values ​​together. Towards class 1 due to the imbalance in the data; So we will get the threshold value from quartiles upwards. You can examine the finishing results by increasing or decreasing the cartridge percentages.

Örnek :

- First threshold  = df.loc[df.Age > 40.5 ,"Age"]
- Second threshold = df.loc[df.Age > 40.5 ,"Age"].quantile(0.90)

Classification achievements corresponding to various quartile values:

- Classification success when 20: 70.49
- Classification success when 25: 71.54
- Classification success at 30: 72.86
- Classification success at 50: 76
- Classification success at 60: 77
- Classification success at 80: 78.83
- Classification success at 90: 79.01

In [ ]:
#Finding the quartiles as threshold values.
df.loc[df.Age > 40.5 ,"Age"].quantile(0.90) 
df.loc[df.Balance > 100710.98499999999 ,"Balance"].quantile(0.90) 
df.loc[df.EstimatedSalary > 101052.94 ,"EstimatedSalary"].quantile(0.90)

In [ ]:
#We created a new variable by taking a copy of df.
xx = df.copy()

Determination of classes with threshold values ​​obtained using quartile above. Since we use the "&" operator below, classes have been written by applying all the possibilities of the "&" operator so that some variables do not appear as empty classes.

In [ ]:
# We're opening an empty column. Because we will perform the classification process with reference to the threshold values.
xx.Exited = np.NaN

xx.loc[(xx.Age > 92) & (xx.Balance > 250898.09) & (xx.EstimatedSalary > 199992.48) , "Exited" ] = 1
xx.loc[(xx.Age > 92.0) & (xx.Balance > 250898.09) , "Exited"] = 1
xx.loc[(xx.Age > 92.0) & (xx.EstimatedSalary > 199992.48) , "Exited"] = 1
xx.loc[(xx.Balance > 250898.09) & (xx.EstimatedSalary > 199992.48) , "Exited"] = 1

xx.loc[(xx.Age <= 92.0) & (xx.Balance <= 250898.09) & (xx.EstimatedSalary <= 199992.48) , "Exited"] = 0
xx.loc[(xx.Age <= 92.0) & (xx.Balance <= 250898.09) , "Exited"] = 0
xx.loc[(xx.Age <= 92.0) & (xx.EstimatedSalary <= 199992.48) , "Exited"] = 0
xx.loc[(xx.Balance <= 250898.09) & (xx.EstimatedSalary <= 199992.48) , "Exited"] = 0

# Assigned class values ​​come in the form of float.
#For the score function, we need to convert it to a categorical variable.

xx.Exited =  xx.Exited.astype('category')
xx.dtypes

In [ ]:
#Searching for missing values
xx.isnull().sum()

#Total number of observations in the data set
len(xx)

In [ ]:
#Comparison of the target variable created using threshold values ​​and the actual target variable y_test
confusion_matrix(xx.Exited, df.Exited)
accuracy_score(xx.Exited, df.Exited)

So, classification can be done manually too :)

See you in another article.

linkedin : https://www.linkedin.com/in/mehmet-kele%C5%9F-531032174/